# Imports & Connections

In [1]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

In [4]:
DB = {'servername': 'DESKTOP-IT4OHPV\SQLEXPRESS',
    'database': 'johari_Herkansing2'}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
conn = ssms_conn.cursor()
conn

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Noah\AppData\Local\Temp\ipykernel_164528\1463935337.py:1: SyntaxWarning: invalid escape sequence '\S'
  DB = {'servername': 'DESKTOP-IT4OHPV\SQLEXPRESS',


In [3]:
# Lokale connecties
sqlite_conn = sqlite3.connect('../data/processed/dwh.sqlite')
go_sales_conn = sqlite3.connect('../data/raw/go_sales.sqlite')
go_staff_conn = sqlite3.connect('../data/raw/go_staff.sqlite')
go_crm_conn = sqlite3.connect('../data/raw/go_crm.sqlite')

# Surrogate Key Creation

In [ ]:
def add_surrogate_key(conn, table_name):
    cursor = conn.cursor()
    cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN SK INT IDENTITY(1,1) PRIMARY KEY;")
    conn.commit()

# Foreign Surrogate Key Creation

In [ ]:
def populate_foreign_keys(conn, fact_table, dim_table, natural_key, foreign_key_column):
    cursor = conn.cursor()
    # This SQL statement assumes that the natural_key uniquely identifies records in the dimension table
    sql = f"""
    UPDATE {fact_table}
    SET {foreign_key_column} = (
        SELECT SK
        FROM {dim_table}
        WHERE {dim_table}.{natural_key} = {fact_table}.{natural_key}
    )
    """
    cursor.execute(sql)
    conn.commit()

# Simulate Slowly Changing Dimensions Type 2

In [ ]:
# Example usage
add_surrogate_key(conn, 'dim_customer')
add_surrogate_key(conn, 'dim_product')

# Assuming 'customer_id' and 'product_id' are natural keys in their respective dimension tables
# and 'customer_sk' and 'product_sk' are the columns in the fact table to hold the foreign surrogate keys
populate_foreign_keys(conn, 'fact_sales', 'dim_customer', 'customer_id', 'customer_sk')
populate_foreign_keys(conn, 'fact_sales', 'dim_product', 'product_id', 'product_sk')